## Part 5
### Implementation of a second order HMM and decoding with Viterbi

In [19]:
def load_original_train():
    with open("train", encoding="utf-8") as file:
        train_list = file.readlines()
        train_list = [x.strip() for x in train_list]
        listToBeReturned = []
        for word_tag_pair in train_list:
            
            if word_tag_pair != "":
                word_tag_pair_list = word_tag_pair.split(" ")
                lower_word = word_tag_pair_list[0].lower()
                listToBeReturned.append(lower_word + " " + word_tag_pair_list[len(word_tag_pair_list)-1])
            else:
                listToBeReturned.append("")
#         print(listToBeReturned)
        return listToBeReturned
    
train_list = load_original_train()

In [20]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in dev_in_list]
        
        return dev_in_list
    
def load_modified_train_file():
    with open("modified_train", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        listToBeReturned = []
        for word_tag_pair in train_list:
            
            if word_tag_pair != "":
                word_tag_pair_list = word_tag_pair.split(" ")
                lower_word = word_tag_pair_list[0].lower()
                listToBeReturned.append(lower_word + " " + word_tag_pair_list[len(word_tag_pair_list)-1])
            else:
                listToBeReturned.append("")
        return listToBeReturned
        
def split_into_sentences(dev_in_list):
    sentences = []
    each_sentence = []
    for word in dev_in_list:
        if word == "":
            sentences.append(each_sentence)
            each_sentence = []
        else:
            each_sentence.append(word)
        
    return sentences

modified_train_list = load_modified_train_file()   
dev_in_list = load_files()
sentences = split_into_sentences(dev_in_list)

In [21]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(train_list)):
    annotated_word = train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


In [22]:
def count_word_tag_pair_dict():
    word_tag_pair_dict= {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair_dict.get(word_tag_pair) == None:
            word_tag_pair_dict[word_tag_pair] = 1
        else:
            word_tag_pair_dict[word_tag_pair] += 1
    return word_tag_pair_dict

def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    

def store_estimate_emission_fix():
    giantEmissionDict = {}
    for word_tag_pair in modified_train_list:
        if word_tag_pair != "":
            word_tag_pair_list = word_tag_pair.split(" ")
            countTop = wordTagPairDict[word_tag_pair]
            countBottom = annotationDict[word_tag_pair_list[len(word_tag_pair_list)-1]]
            emission = countTop/countBottom
            if giantEmissionDict.get(word_tag_pair) == None:
                giantEmissionDict[word_tag_pair] = countTop/countBottom
    return giantEmissionDict

def store_estimate_transition():
    giantTransitionDict = {}
    finalGiantTransitionDict = {}
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if giantTransitionDict.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                giantTransitionDict[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
    for eachTransition in giantTransitionDict:
        eachTransitionList = eachTransition.split(" ")
        startState = eachTransitionList[0]
        countTop = giantTransitionDict[eachTransition]
        countBottom = totalStateNumber[startState]
        transitionParam = countTop/countBottom
        finalGiantTransitionDict[eachTransition] = transitionParam
        
    return finalGiantTransitionDict

def separate_tweets():
    countStart = 1
    # counting starts
    giant_each_tweet = []
    each_tweet = []
    totalStateNumber = {}
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giant_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[len(each_word)-1])
    for i in range(len(giant_each_tweet)):
        giant_each_tweet[i].insert(0, 'START')
        giant_each_tweet[i].append('STOP')
    
    for each_tweet in giant_each_tweet:
        for tag in each_tweet:
            if totalStateNumber.get(tag) == None:
                totalStateNumber[tag] = 1
            else:
                totalStateNumber[tag] += 1

    return giant_each_tweet, countStart, totalStateNumber

modifiedWordDict = get_word_dict()
wordTagPairDict = count_word_tag_pair_dict()
giantEmissionDict = store_estimate_emission_fix()
giantTaggedEachTweet, countStart, totalStateNumber = separate_tweets()
giantTransitionDict = store_estimate_transition()

In [23]:
def store_second_order_transition():
    transitionCount = {}
    startStateCount = {}
    giantSecondTransitionDict = {}        
    for taggedEachTweet in giantTaggedEachTweet:
        for i in range(len(taggedEachTweet) - 1):
            if startStateCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1]) == None:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] = 1
            else:
                startStateCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]] += 1
                
        for i in range(len(taggedEachTweet) - 2):
            if transitionCount.get(taggedEachTweet[i]+" "+taggedEachTweet[i+1] + " " + taggedEachTweet[i+2]) == None:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] = 1
            else:
                transitionCount[taggedEachTweet[i]+" "+taggedEachTweet[i+1]+ " "+ taggedEachTweet[i+2]] += 1
    for eachTransition in transitionCount:
        eachTransitionList = eachTransition.split(" ") #START O O
        startState = eachTransitionList[0] + " " + eachTransitionList[1] # START O
        countTop = transitionCount[eachTransition]
        countBottom = startStateCount[startState]
        transitionParam = countTop/countBottom
        giantSecondTransitionDict[eachTransition] = transitionParam

    return giantSecondTransitionDict

giantSecondTransitionDict = store_second_order_transition()
print(giantSecondTransitionDict)

{'START O O': 0.7756956274843839, 'O O O': 0.8720445062586927, 'O O STOP': 0.07620737614502901, 'START B-neutral O': 0.9, 'B-neutral O O': 0.9019607843137255, 'START O B-positive': 0.16524701873935263, 'O B-positive O': 0.6900532859680284, 'B-positive O O': 0.7884615384615384, 'O O B-negative': 0.013668409189007722, 'O B-negative O': 0.7817679558011049, 'B-negative O STOP': 0.12374581939799331, 'O O B-positive': 0.0365929691621505, 'B-positive O STOP': 0.15264423076923078, 'START O B-negative': 0.04031800113571834, 'B-negative O O': 0.862876254180602, 'O B-negative I-negative': 0.20994475138121546, 'B-negative I-negative I-negative': 0.3875, 'I-negative I-negative O': 0.5849056603773585, 'I-negative O O': 0.8, 'O O B-neutral': 0.001486739245120138, 'O B-neutral O': 0.7636363636363637, 'O B-positive I-positive': 0.29573712255772644, 'B-positive I-positive O': 0.6083333333333333, 'I-positive O O': 0.7549295774647887, 'START O B-neutral': 0.01306076093128904, 'B-positive I-positive I-posi

In [24]:
def backtrack_second_order(piList, sentence):
    print(piList)
    print(sentence)
    path = []
    singleStringToBeWritten = ""
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
        
    if len(piList) == len(sentence):
        for i in range(len(piList)-1):
            if i == 0:
                lastLayer = piList[len(piList)- (i+1)]
                maxPiTag = max(lastLayer, key=lastLayer.get)
                maxPiTagList = maxPiTag.split(" ")
                path.append(maxPiTagList[len(maxPiTagList)-1])
                path.append(maxPiTagList[0])

            else:
                currentLayer = piList[len(piList) - (i+1)] #2nd last layer onwards
                targetedTag = path[i-1]
                scoreDict = {}
                for tags in currentLayer:
                    if giantSecondTransitionDict.get(tags + " " + targetedTag) != None:
                        scoreOfTag = currentLayer[tags]*giantSecondTransitionDict[tags + " " + targetedTag]
                        scoreDict[tags] = scoreOfTag
                bestScoreTag = max(scoreDict, key=scoreDict.get)
                bestScoreTagList = bestScoreTag.split(" ")
                path.append(bestScoreTagList[0])
                if bestScoreTagList[0] == "START":
                    path.append(bestScoreTagList[len(bestScoreTagList)-1])
    path.remove("STOP")
    path.remove("START")
    for j in range(len(sentence)):
        if sentence[j] == "":
            singleStringToBeWritten += "\n"
        else:
            singleStringToBeWritten += sentence[j] + " " + path[len(path)-(j+1)] + "\n"
    return singleStringToBeWritten
piList = [{'O': 0.10952242946302841, 'B-negative': 0.0019577571933592874, 'B-positive': 0.010624504726325918, 'B-neutral': 0.0009040105193951349}, {'START O': 0.00024221306515862052, 'START B-negative': 0.0001000958565025801, 'START B-positive': 0.0012160526263888527, 'START B-neutral': 1.9652402595546413e-05}, {'O B-negative': 2.5578783356529717e-08, 'O B-positive': 9.94562791068425e-08}, {'B-positive STOP': 1.413233095656732e-09}]
sentence = ['AVOID', 'THAT', 'PLACE', '']
piList2 = [{'B-positive': 0.00014504443312390332, 'B-neutral': 8.21827744904668e-05}, {'START B-positive': 1.701560172955939e-06}, {'B-positive O': 7.4384900942866305e-09}, {'O O': 2.2814373537614629e-10}, {'O O': 1.3952466906739993e-13}, {'O O': 2.5096579686371738e-17}, {'O O': 1.375919532845953e-18}, {'O STOP': 1.0482944960703226e-19}]
sentence2 = ['lobster', 'was', 'good', ',', 'nothing', 'spectacular', '.', '']
# backtrack_second_order(piList, sentence)
# backtrack_second_order(piList2, sentence2)

In [25]:
def second_order_viterbi(sentence):
    all_tags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
    sentence.append("") #so we can detect the end
    piList = []
    for i in range(len(sentence)):
        word = sentence[i].lower()
        if word in modifiedWordDict:
            word = word
        else:
            word = "#UNK#"
            
        piLayer = {}
        if i == 0:
            for tag in all_tags:
                if giantTransitionDict.get("START" + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                    pi = giantTransitionDict["START" + " " + tag]*giantEmissionDict[word + " " + tag]
                    piLayer[tag] = pi
            piList.append(piLayer)
        
        elif i == 1:
            previousLayer = piList[len(piList)-1]
            for tag in all_tags:
                tempLayer = {}
                for previous_tag in previousLayer:
                    if giantSecondTransitionDict.get("START" + " " + previous_tag + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                        pi = previousLayer[previous_tag]*giantSecondTransitionDict["START" + " " + previous_tag + " " + tag]*giantEmissionDict[word + " " + tag]
                        tempLayer["START" + " " + previous_tag] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get) 
                    maxPiScore = max(tempLayer.values())
                    piLayer[maxPiTag] = maxPiScore
            piList.append(piLayer)
            
        else:
            
            previousLayer = piList[len(piList)-1]
            if word == "":
#                 print(piList)
                tempLayer = {}
                for previous_tags in previousLayer:
                    previous_tags_list = previous_tags.split(" ")
                    if giantSecondTransitionDict.get(previous_tags + " " + "STOP") != None:
                        pi = previousLayer[previous_tags]*giantSecondTransitionDict[previous_tags + " " + "STOP"]
                        tagOneLayerBefore = previous_tags_list[len(previous_tags_list)-1]
                        tempLayer[tagOneLayerBefore+ " " + "STOP"] = pi
                if len(tempLayer) != 0:
                    maxPiTag = max(tempLayer, key=tempLayer.get) 
                    maxPiScore = max(tempLayer.values())
                    piLayer[maxPiTag] = maxPiScore
                piList.append(piLayer)
                singleStringToBeWritten = backtrack_second_order(piList, sentence)
#                 print(singleStringToBeWritten)
                return singleStringToBeWritten
            else:
                for tag in all_tags:
                    tempLayer = {}
                    for previous_tags in previousLayer:
                        previous_tags_list = previous_tags.split(" ")
                        if giantSecondTransitionDict.get(previous_tags + " " + tag) != None and giantEmissionDict.get(word + " " + tag) != None:
                            pi = previousLayer[previous_tags]*giantSecondTransitionDict[previous_tags + " " + tag]*giantEmissionDict[word + " " + tag]
                            tagOneLayerBefore = previous_tags_list[len(previous_tags_list)-1]
                            tempLayer[tagOneLayerBefore+ " " + tag] = pi
                    if len(tempLayer) != 0:
                        maxPiTag = max(tempLayer, key=tempLayer.get) 
                        maxPiScore = max(tempLayer.values())
                        piLayer[maxPiTag] = maxPiScore
                piList.append(piLayer)
    
                        

            
giantStringToBeWritten = ""    
for sentence in sentences:
    singleStringToBeWritten = second_order_viterbi(sentence)
    print(singleStringToBeWritten)
    giantStringToBeWritten += singleStringToBeWritten
print(giantStringToBeWritten)
f = open("dev.p5.out","w+")
f.write(giantStringToBeWritten)
# second_order_viterbi(["The", "tuna", "and", "wasabe", "potatoes", "are", "excellent", "."])
# second_order_viterbi(["lobster", "was", "good", ",", "nothing","spectacular","."])

[{'O': 0.0020167704776798356}, {'START O': 3.342790286812497e-05}, {'O O': 7.487383327920167e-09}, {'O O': 5.252122935344911e-11}, {'O O': 3.778636211503018e-15}, {'O O': 1.281790293406509e-16}, {'O O': 2.388454331631745e-18}, {'O O': 1.7183718161449065e-21}, {'O O': 8.035829191372904e-25}, {'O O': 1.4973752835480763e-26}, {'O O': 3.070263993072444e-29}, {'O O': 1.148627638742517e-31}, {'O O': 1.2395690398098006e-35}, {'O O': 2.158173152819642e-37}, {'O O': 4.037010071632344e-40}, {'O O': 3.862884056366419e-42}, {'O O': 1.584115078963055e-44}, {'O O': 1.1396904973940162e-48}, {'O O': 3.866062079752869e-50}, {'O O': 2.837064401826265e-52}, {'O O': 5.286511213664771e-54}, {'O O': 8.13922642592514e-56}, {'O O': 8.783641230001969e-60}, {'O O': 1.5292910743800646e-61}, {'O O': 2.8606432536072368e-64}, {'O O': 2.7372567863927595e-66}, {'O O': 9.285332016557717e-68}, {'O O': 6.680325694107009e-72}, {'O O': 3.2201243176705052e-74}, {'O O': 8.108506742373607e-78, 'O B-positive': 9.7544627363251

ValueError: max() arg is an empty sequence